# Image classification distributed training

**Requirements** - In order to benefit from this tutorial, you will need:
- A basic understanding of Machine Learning
- An Azure account with an active subscription - [Create an account for free](https://azure.microsoft.com/free/?WT.mc_id=A261C142F)
- An Azure ML workspace with computer cluster - [Configure workspace](../../configuration.ipynb)
- A python environment
- Installed Azure Machine Learning Python SDK v2 - [install instructions](../../../README.md) - check the getting started section

**Learning Objectives** - By the end of this tutorial, you should be able to:
- Connect to your AML workspace from the Python SDK
- Create `Pipeline` with components

**Motivations** -This is a sample to demonstrate distributed training in azure machine learning. In this pipeline, we use components to do image preprocessing on cpu nodes, and a mpi custom component on distributed gpu nodes.

# 1. Connect to Azure Machine Learning Workspace

The [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.

## 1.1 Import the required libraries

In [ ]:
# import required libraries
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential

from azure.ai.ml import MLClient, Input
from azure.ai.ml.dsl import pipeline
from azure.ai.ml import load_component

## 1.2 Configure credential

We are using `DefaultAzureCredential` to get access to workspace. 
`DefaultAzureCredential` should be capable of handling most Azure SDK authentication scenarios. 

Reference for more available credentials if it does not work for you: [configure credential example](../../configuration.ipynb), [azure-identity reference doc](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity?view=azure-python).

In [ ]:
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

## 1.3 Get a handle to the workspace

We use config file to connect to a workspace. The Azure ML workspace should be configured with computer cluster. [Check this notebook for configure a workspace](../../configuration.ipynb)

In [ ]:
# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential)

# Retrieve an already attached Azure Machine Learning Compute.
cpu_compute_target = "cpu-cluster"
print(ml_client.compute.get(cpu_compute_target))
gpu_compute_target = "gpu-cluster"
print(ml_client.compute.get(gpu_compute_target))

## Prepare dataset
This dataset is a subset of the [Open Images Dataset](https://storage.googleapis.com/openimages/web/index.html).
- training dataset contains 24 images (3 categories * 8 images per category)
- validation dataset contains 6 images (3 categories * 2 images per category)

This is a extremely small dataset only for demo usage for this notebook. Use larger datasets to train models for production usage. 

NOTE: Use zip file here to avoid performance issue of mounting file dataset with many subfolders.

In [ ]:
train_image_dataset = Input(name="TrainData", type="uri_folder", path="./data/train")
val_image_dataset = Input(name="ValidData", type="uri_folder", path="./data/val")

# 2. Define command component via YAML
Below is a basic example to define command component  using YAML.


In [ ]:
apply_transform = load_component(
    path="./apply_image_transformation/apply_image_transformation.yaml"
)
convert = load_component(
    path="./convert_to_image_directory/convert_to_image_directory.yaml"
)
init_transform = load_component(
    path="./init_image_transformation/init_image_transformation.yaml"
)

# this train component is an mpi component.
imagecnn_train = load_component(source="./imagecnn_train/entry.spec.yaml")

# 3. Basic pipeline job

## 3.1 Build pipeline

In [ ]:
# define pipeline
@pipeline(
    default_compute=cpu_compute_target,
)
def image_classification_with_densenet():
    """E2E image classification pipeline with densenet."""
    convert_train = convert(input_path=train_image_dataset)

    convert_val = convert(input_path=val_image_dataset)

    init_image_transformation = init_transform(
        resize="False",
        size=256,
        center_crop="False",
        crop_size=224,
        pad="False",
        padding=0,
        color_jitter="False",
        grayscale="False",
        random_resized_crop="False",
        random_resized_crop_size=256,
        random_crop="False",
        random_crop_size=224,
        random_horizontal_flip="True",
        random_vertical_flip="False",
        random_rotation="False",
        random_rotation_degrees=0,
        random_affine="False",
        random_affine_degrees=0,
        random_grayscale="False",
        random_perspective="False",
    )

    apply_trans_on_train = apply_transform(
        mode="For training",
        input_image_transform_path=init_image_transformation.outputs.output_path,
        input_image_dir_path=convert_train.outputs.output_path,
    )

    apply_trans_on_val = apply_transform(
        mode="For inference",
        input_image_transform_path=init_image_transformation.outputs.output_path,
        input_image_dir_path=convert_val.outputs.output_path,
    )

    imagecnn_train_gpu = imagecnn_train(
        train_data=apply_trans_on_train.outputs.output_path,
        valid_data=apply_trans_on_val.outputs.output_path,
        data_backend="pytorch",
        arch="resnet50",
        model_config="classic",
        workers=5,
        epochs=4,
        batch_size=16,
        optimizer_batch_size=-1,
        lr=0.1,
        lr_schedule="step",
        warmup=0,
        label_smoothing=0.0,
        mixup=0.0,
        momentum=0.9,
        weight_decay=0.0001,
        print_freq=10,
        resume="",
        pretrained_weights="",
        static_loss_scale=1.0,
        prof=-1,
        seed=123,
        raport_file="experiment_raport.json",
        save_checkpoint_epochs=2,
    )
    imagecnn_train_gpu.compute = gpu_compute_target
    imagecnn_train_gpu.resources.instance_count = 1


# create a pipeline
pipeline_job = image_classification_with_densenet()

In [ ]:
print(pipeline_job)

# 3.2 Submit pipeline job

In [ ]:
pipeline_job = ml_client.jobs.create_or_update(
    pipeline_job, experiment_name="pipeline_samples"
)
pipeline_job

In [ ]:
# Wait until the job completes
ml_client.jobs.stream(pipeline_job.name)

# Next Steps
You can see further examples of running a pipeline job [here](../)